### Wstęp do Uczenia Maszynowego 
##### Laboratorium 03

### 1. Kroswalidacja (*ang. cross validation*) vs Leave One Out

![](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

https://scikit-learn.org/stable/modules/cross_validation.html

-----
##### *Zadanie 1*
------
Wymień zalety i wady kroswalidacji.


Zalety:
- dobrze działa w praktyce
- używamy wszystkich danych jako zbiór treningowy i testowy

Wady:
- zbiór treningowy i testowy nie są niezależne 
- możemy policzyć odchylenie standardowe, ale nie ma gwarancji na nie
- błąd jest obarczony błędem (bias) - próbki treningowe są mniejsze niż pełny zbiór
- wydłużenie czasu budowy, $k$ razy trzeba trenowac modele

----------------------------------------------------------------------------------

In [ ]:
# Import potrzebnych bibliotek i modułów
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [ ]:
# Wczytanie danych
pima = pd.read_csv("../lab02/pima.csv")


In [ ]:
# Definicja zbioru X i y
y = pima.Outcome
X = pima.drop(["Outcome"], axis = 1)

-----
##### *Zadanie 1.5*
------

Dla danych `pima` zbadaj zachowanie hiperparametru modelu drzewa klasyfikacyjnego `max_leaf_nodes` w zakresie od 2 do 30 na zbiorze treningowym i testowym. Oblicz miarę dokładości, precyzji i AUC. Przygotuj wizualizacje.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score
acc_train = [] 
acc_test = [] 
auc_train = [] 
auc_test = [] 
prec_train = [] 
prec_test = [] 

for i in range(2, 31):
    Tree_i = tree.DecisionTreeClassifier(max_leaf_nodes=i)
    # Uczenie modelu na danych treningowych
    Tree_i.fit(X_train, y_train)
    
    y_train_pred = Tree_i.predict(X_train) # Predykcje modelu dla danych treningowych (klasy)
    y_train_pred_proba = Tree_i.predict_proba(X_train)[:,1] # Predykcje modelu dla danych treningowych (prawdopodobieństwa)
    acc_train.append(accuracy_score(y_train, y_train_pred)) # Wyliczenie dokładności dla zbioru treningowego
    auc_train.append(roc_auc_score(y_train, y_train_pred_proba)) # Wyliczenie AUC dla zbioru treningowego
    prec_train.append(precision_score(y_train, y_train_pred)) # Wyliczenie precyzji dla zbioru treningowego
    
    y_test_pred = Tree_i.predict(X_test) # Predykcje modelu dla danych testowych (klasy)
    y_test_pred_proba = Tree_i.predict_proba(X_test)[:,1] # Predykcje modelu dla danych testowych (prawdopodobieństwa)
    acc_test.append(accuracy_score(y_test, y_test_pred)) # Wyliczenie dokładności dla zbioru testowego
    auc_test.append(roc_auc_score(y_test, y_test_pred_proba)) # Wyliczenie AUC dla zbioru testowego
    prec_test.append(precision_score(y_test, y_test_pred)) # Wyliczenie precyzji dla zbioru testowego
    


In [ ]:
import matplotlib.pyplot as plt
max_leaf_nodes_range = np.arange(2, 31)
plt.figure(figsize=(15, 5))

# Dokładność
plt.subplot(1, 3, 1)
plt.plot(max_leaf_nodes_range, acc_train, label='Trening')
plt.plot(max_leaf_nodes_range, acc_test, label='Test')
plt.title('Accuracy (Dokładność)')
plt.xlabel('max_leaf_nodes')
plt.ylabel('Accuracy Score')
plt.legend()

# AUC
plt.subplot(1, 3, 2) 
plt.plot(max_leaf_nodes_range, auc_train, label='Trening')
plt.plot(max_leaf_nodes_range, auc_test, label='Test')
plt.title('ROC AUC Score')
plt.xlabel('max_leaf_nodes')
plt.ylabel('AUC Score')
plt.legend()

# Precyzja
plt.subplot(1, 3, 3) 
plt.plot(max_leaf_nodes_range, prec_train, label='Trening')
plt.plot(max_leaf_nodes_range, prec_test, label='Test')
plt.title('Precision (Precyzja)')
plt.xlabel('max_leaf_nodes')
plt.ylabel('Precision Score')
plt.legend()

#

-----
##### *Zadanie 2*
------
Używając funkcji `cross_val_score()` oszacuj wartość średnią oraz odchylenie standardowe 10-krotnej kroswalidacji dla modelu drzewa.

In [ ]:
# Podział zbioru na próbkę treningową i testową
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
# Budowa modelu drzewa używając kroswalidacji na zbiorze treningowym
Tree = tree.DecisionTreeClassifier()
scores = cross_val_score(Tree, X_train, y_train, cv = 10)

In [ ]:
scores

In [ ]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

-----
##### *Zadanie 3*
------
Przeprowadź na zbiorze danych `pima.csv` 100 razy 10-krotną kroswalidację i przestaw otrzymane wyniki (miara `roc_auc`) - średnią oraz odchylenie standardowe. Narysuj rozkład otrzymanych wyników.



In [ ]:
# Budowa drzewa
Tree = tree.DecisionTreeClassifier()
scores_all = []
for i in range(0, 100):
    # Kroswalidacja 10-krotna (parametr "cv") z miarą roc_auc (parametr "scoring")
    scores = cross_val_score(Tree, X_train, y_train, cv = 10, scoring = 'roc_auc')
    scores_all.append(scores)


In [ ]:
# Ramka danych tworzona z listy list
df = pd.DataFrame(scores_all)

In [ ]:
# Wyliczenie średniej i narysowanie histogramu
df.aggregate(["mean", "std"], axis=1)["mean"].hist()

In [ ]:
# Wyliczenie odchylenia standardowego i narysowanie histogramu
df.aggregate(["mean", "std"], axis=1)["std"].hist()

### 2. Krzywa ROC

Jak się wylicza krzywą ROC?

1. Potrzebujemy wyliczyć FPR (false positive ratio) i TPR (true positive ratio) dla różnych rozważanych progów odcięcia.

$FPR = \frac{FP}{FP + TN}$

$TPR = \frac{TP}{TP + FN}$

2. Przykład

$y = [0, 0, 1, 1]$

$y_{pred} = [0.1, 0.4, 0.35, 0.8]$

$treshold = [0.1, 0.35, 0.4, 0.8]$

-----
##### *Zadanie 5*
------
Przygotuj model do oceny metodą `roc_auc`.


In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

In [ ]:
# Trzy różne modele drzewa.
Tree1 = tree.DecisionTreeClassifier()
Tree2 = tree.DecisionTreeClassifier(max_depth=3)
Tree3 = tree.DecisionTreeClassifier(max_leaf_nodes=5)

In [ ]:
# Trening modeli
Tree1 = Tree1.fit(X_train, y_train)
Tree2 = Tree2.fit(X_train, y_train)
Tree3 = Tree3.fit(X_train, y_train)

In [ ]:
# Predykcja modeli - prawdopodobieństwa
pred1 = Tree1.predict_proba(X_test)
pred2 = Tree2.predict_proba(X_test)
pred3 = Tree3.predict_proba(X_test)

In [ ]:
# Wyliczanie krzywej ROC dla 3 modeli
fpr, tpr, thresholds = roc_curve(y_test, pred1[:,1])
plt.plot(fpr,tpr,label="Tree, AUC="+str(round(roc_auc_score(y_test, pred1[:,1]), 4)))
fpr, tpr, thresholds = roc_curve(y_test, pred2[:,1])
plt.plot(fpr,tpr,label="Tree, max_depth=3, AUC="+str(round(roc_auc_score(y_test, pred2[:,1]), 4)))
fpr, tpr, thresholds = roc_curve(y_test, pred3[:,1])
plt.plot(fpr,tpr,label="Tree, max_leaf_nodes=5, AUC="+str(round(roc_auc_score(y_test, pred3[:,1]), 4)))
plt.title('ROC AUC')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.legend()

In [ ]:
# Drugi sposób rysowanie krzywej ROC
from sklearn.metrics import RocCurveDisplay
RocCurveDisplay.from_estimator(Tree1, X_test, y_test)

In [ ]:
# Wyliczenia miary AUC.
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, Tree1.predict_proba(X_test)[:, 1])

### 3. Podsumowanie, czyli co warto wiedzieć...

1. Jak definiujemy problem klasyfikacji?
2. Podział zbioru na treningowy, walidacyjny i testowy.
2. Miary dokładności klasyfikatorów:

    a) macierz pomyłek (*ang. confusion matrix*),

    b) dokładność, czułość, precyzja, (*ang. accuracy, recall, precision*)

    c) krzywa ROC,

    d) AUC.


3. Kroswalidacja (*ang. cross validation*)
4. Drzewa decyzyjne:

    a) wady i zalety,

    b) jak budujemy drzewo,

    b) jakie mamy miary podziału,
    
    d) przycinanie drzew.